In [1]:
using JuMP, Gurobi
using DataFrames, CSV, Combinatorics
using Random
using LinearAlgebra
using Distances
using Plots, StatsPlots
using Suppressor
using Pkg
Pkg.add("StatsPlots")
Pkg.add("Random")
Pkg.add("Combinatorics")
Pkg.add("Distances")
Pkg.add("Suppressor")
area = CSV.read("Pb1_areas.csv", DataFrame);
shelter = CSV.read("Pb1_shelters.csv", DataFrame);
area = Matrix(area)
shelter = Matrix(shelter); 

# assumed variable
I = size(area, 1)
J = size(shelter, 1)

# demand
pop = area[:,3]
totpop = sum(area[:,3])

# supply
capa = shelter[:, 3]

# supply-demand
dist = zeros(I,J);
for i = 1:I, j = 1:J
    dist[i,j] =  abs(area[i,1]-shelter[j,1]) +  abs(area[i,2]-shelter[j,2])
end

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
# Definition of model
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 60); 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11
Set parameter TimeLimit to value 60


In [3]:
@variable(model, r[1:I, 1:J], Bin);
@variable(model, b[1:J], Bin);

55-element Vector{VariableRef}:
 b[1]
 b[2]
 b[3]
 b[4]
 b[5]
 b[6]
 b[7]
 b[8]
 b[9]
 b[10]
 ⋮
 b[47]
 b[48]
 b[49]
 b[50]
 b[51]
 b[52]
 b[53]
 b[54]
 b[55]

In [4]:
@constraint(
    model,
    supply_atmost10,
    sum(b[j] for j in 1:J) <= 10
);

In [5]:
@constraint(
    model, 
    supply_ifbuilt_twostone[j in 1:J],
    sum(pop[i] * r[i,j] for i in 1:I) <= capa[j] * b[j]
);

In [6]:
# 1 (sum) + J (ineq) + I (equal) = 1 + 55 + 200 
@constraint(
    model, 
    demand_area_partition[i in 1:I],
    sum(r[i,j] for  j in 1:J) == 1
);

In [7]:
model

A JuMP Model
Feasibility problem with:
Variables: 11055
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 200 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 56 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 11055 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: b, demand_area_partition, r, supply_atmost10, supply_ifbuilt_twostone

In [8]:
@objective(
    model, 
    Min,
    sum(
        dist[i,j] * r[i,j] * pop[j]
        for i in 1:I,
            j in 1:J
    )
);

In [10]:
for j in 1:J
    if value(r[125,j]) > 0.001
         println("area 125 send to $j  = ", value(r[125,j]))
        #end
    end
end

area 125 send to 52  52 = 1.0


In [12]:
value.(b)

55-element Vector{Float64}:
  1.0
 -0.0
 -0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.0
  ⋮
  1.0
  0.0
  1.0
  1.0
  0.0
  1.0
  0.0
  0.0
  0.0

In [9]:
optimize!(model)

Set parameter TimeLimit to value 60
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 256 rows, 11055 columns and 22110 nonzeros
Model fingerprint: 0x6007bceb
Variable types: 0 continuous, 11055 integer (11055 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.02s
Presolved: 256 rows, 11055 columns, 22110 nonzeros
Variable types: 0 continuous, 11055 integer (11055 binary)

Root relaxation: objective 8.945118e+04, 253 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 89451.1829    0   50          - 89451.1829      -     -    0s
H    0     0                    719100.9214

In [11]:
objective_value(model)  / totpop

2.022713988940276